In [3]:
%%pyspark
import pyspark 
print(print(pyspark.__version__)) 

2.4.4.dev0
None

In [4]:
import azureml.core
print(azureml.core.VERSION)

1.0.69

In [5]:
import os

subscription_id = "c46a9435-c957-4e6c-a0f4-b9a597984773"
resource_group = "eastus_automl_rg"
workspace_name = "eastus_automl_ws"
workspace_region = "East US 2"

In [6]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

Workspace configuration succeeded. Skip the workspace creation steps below

In [7]:
import azureml.core
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpucluster

In [9]:
import azureml.dataprep as dprep

In [10]:
dataset_root = "https://dprepdata.blob.core.windows.net/demo"

green_path = "/".join([dataset_root, "green-small/*"])
yellow_path = "/".join([dataset_root, "yellow-small/*"])

green_df = dprep.read_csv(path=green_path, header=dprep.PromoteHeadersMode.GROUPED)
# auto_read_file will automatically identify and parse the file type, and is useful if you don't know the file type
yellow_df = dprep.auto_read_file(path=yellow_path)

green_df.head(5)
yellow_df.head(5)

vendor_name         ...                   Total_Amt
0                     ...                          NA
1         VTS         ...          9.4000000000000004
2         VTS         ...                        14.6
3         VTS         ...          28.440000000000001
4         DDS         ...          18.449999999999999

[5 rows x 18 columns]

In [11]:
all_columns = dprep.ColumnSelector(term=".*", use_regex=True)
drop_if_all_null = [all_columns, dprep.ColumnRelationship(dprep.ColumnRelationship.ALL)]
useful_columns = [
    "cost", "distance", "dropoff_datetime", "dropoff_latitude", "dropoff_longitude",
    "passengers", "pickup_datetime", "pickup_latitude", "pickup_longitude", "store_forward", "vendor"
]

In [12]:
tmp_df = (green_df
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "VendorID": "vendor",
        "lpep_pickup_datetime": "pickup_datetime",
        "Lpep_dropoff_datetime": "dropoff_datetime",
        "lpep_dropoff_datetime": "dropoff_datetime",
        "Store_and_fwd_flag": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Pickup_longitude": "pickup_longitude",
        "Pickup_latitude": "pickup_latitude",
        "Dropoff_longitude": "dropoff_longitude",
        "Dropoff_latitude": "dropoff_latitude",
        "Passenger_count": "passengers",
        "Fare_amount": "cost",
        "Trip_distance": "distance"
     })
    .keep_columns(columns=useful_columns))
tmp_df.head(5)

vendor      pickup_datetime  ...   distance   cost
0      2  2013-08-01 08:14:37  ...        .00  21.25
1      2  2013-08-01 09:13:00  ...        .00   74.5
2      2  2013-08-01 09:48:00  ...        .00      1
3      2  2013-08-01 10:38:35  ...        .00   3.25
4      2  2013-08-01 11:51:45  ...        .00    8.5

[5 rows x 11 columns]

In [13]:
green_df = tmp_df

In [14]:
tmp_df = (yellow_df
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "vendor_name": "vendor",
        "VendorID": "vendor",
        "vendor_id": "vendor",
        "Trip_Pickup_DateTime": "pickup_datetime",
        "tpep_pickup_datetime": "pickup_datetime",
        "Trip_Dropoff_DateTime": "dropoff_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "store_and_forward": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Start_Lon": "pickup_longitude",
        "Start_Lat": "pickup_latitude",
        "End_Lon": "dropoff_longitude",
        "End_Lat": "dropoff_latitude",
        "Passenger_Count": "passengers",
        "passenger_count": "passengers",
        "Fare_Amt": "cost",
        "fare_amount": "cost",
        "Trip_Distance": "distance",
        "trip_distance": "distance"
    })
    .keep_columns(columns=useful_columns))
tmp_df.head(5)

vendor         ...                        cost
0    VTS         ...          8.9000000000000004
1    VTS         ...                        12.1
2    VTS         ...          23.699999999999999
3    DDS         ...                        14.9
4    DDS         ...          3.7000000000000002

[5 rows x 11 columns]

In [15]:
yellow_df = tmp_df
combined_df = green_df.append_rows([yellow_df])

In [16]:
decimal_type = dprep.TypeConverter(data_type=dprep.FieldType.DECIMAL)
combined_df = combined_df.set_column_types(type_conversions={
    "pickup_longitude": decimal_type,
    "pickup_latitude": decimal_type,
    "dropoff_longitude": decimal_type,
    "dropoff_latitude": decimal_type
})
combined_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude", 
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

ColumnProfile:
    column_name: pickup_longitude
    type: FieldType.DECIMAL

    min: -115.17933654785156
    max: 0.0
    count: 7821.0
    missing_count: 0.0
    not_missing_count: 7821.0
    percent_missing: 0.0
    error_count: 0.0
    empty_count: 0.0
    unique_values: >1000


    Quantiles (est.):
         0.1%: -80.02003484562465
           1%: -73.96327505334597
           5%: -73.96370363235475
          25%: -73.94862645269899
          50%: -73.92287611248696
          75%: -73.84618838349073
          95%: 0.0
          99%: 0.0
        99.9%: 0.0

    mean: -68.89872787497673
    std: 18.682200424494315
    variance: 349.0246127009756
    skewness: 3.3968830906644167
    kurtosis: 9.65922988524884
    whisker_top: -73.70304616292317
    whisker_bottom: -74.0927963256836

ColumnProfile:
    column_name: pickup_latitude
    type: FieldType.DECIMAL

    min: 0.0
    max: 40.91912078857422
    count: 7821.0
    missing_count: 0.0
    not_missing_count: 7821.0
    percent_mis

In [18]:
tmp_df = (combined_df
    .drop_nulls(
        columns=["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"],
        column_relationship=dprep.ColumnRelationship(dprep.ColumnRelationship.ANY)
    ) 
    .filter(dprep.f_and(
        dprep.col("pickup_longitude") <= -73.72,
        dprep.col("pickup_longitude") >= -74.09,
        dprep.col("pickup_latitude") <= 40.88,
        dprep.col("pickup_latitude") >= 40.53,
        dprep.col("dropoff_longitude") <= -73.72,
        dprep.col("dropoff_longitude") >= -74.09,
        dprep.col("dropoff_latitude") <= 40.88,
        dprep.col("dropoff_latitude") >= 40.53
    )))
tmp_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude", 
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

ColumnProfile:
    column_name: pickup_longitude
    type: FieldType.DECIMAL

    min: -74.07815551757812
    max: -73.73648071289062
    count: 7158.0
    missing_count: 0.0
    not_missing_count: 7158.0
    percent_missing: 0.0
    error_count: 0.0
    empty_count: 0.0
    unique_values: >1000


    Quantiles (est.):
         0.1%: -74.07800495910645
           1%: -73.9638987099072
           5%: -73.96399193619052
          25%: -73.94948645963127
          50%: -73.92936420248408
          75%: -73.86861771130171
          95%: -73.83046139425237
          99%: -73.82445373888983
        99.9%: -73.76963703104654

    mean: -73.91478407411482
    std: 0.04904857906887633
    variance: 0.0024057631086758136
    skewness: 0.40343220430988064
    kurtosis: -0.6164518353378066
    whisker_top: -73.7475700378418
    whisker_bottom: -74.01773607548829

ColumnProfile:
    column_name: pickup_latitude
    type: FieldType.DECIMAL

    min: 40.57548522949219
    max: 40.879852294921875
    

In [19]:
combined_df = tmp_df

In [20]:
combined_df.keep_columns(columns='store_forward').get_profile()

ColumnProfile:
    column_name: store_forward
    type: FieldType.STRING

    min: N
    max: Y
    count: 7158.0
    missing_count: 198.0
    not_missing_count: 6960.0
    percent_missing: 0.027661357921207042
    error_count: 0.0
    empty_count: 0.0
    unique_values: 3

In [21]:
combined_df = combined_df.replace(columns="store_forward", find="0", replace_with="N").fill_nulls("store_forward", "N")

In [22]:
combined_df = combined_df.replace(columns="distance", find=".00", replace_with=0).fill_nulls("distance", 0)
combined_df = combined_df.to_number(["distance"])

In [23]:
tmp_df_renamed = (tmp_df
    .rename_columns(column_pairs={
        "pickup_datetime_1": "pickup_date",
        "pickup_datetime_2": "pickup_time",
        "dropoff_datetime_1": "dropoff_date",
        "dropoff_datetime_2": "dropoff_time"
    }))
tmp_df_renamed.head(5)

vendor      pickup_datetime     dropoff_datetime  ...  passengers  distance  cost
0      2  2013-08-01 17:22:00  2013-08-01 17:22:00  ...           1       .00   2.5
1      2  2013-08-01 17:24:00  2013-08-01 17:25:00  ...           1       .00   2.5
2      2  2013-08-06 06:51:19  2013-08-06 06:51:36  ...           1       .00   3.3
3      2  2013-08-06 13:26:34  2013-08-06 13:26:57  ...           1       .00   3.3
4      2  2013-08-06 13:27:53  2013-08-06 13:28:08  ...           1       .00   3.3

[5 rows x 11 columns]

In [24]:
combined_df = tmp_df_renamed
combined_df.get_profile()

ColumnProfile:
    column_name: vendor
    type: FieldType.STRING

    min: 1
    max: VTS
    count: 7158.0
    missing_count: 0.0
    not_missing_count: 7158.0
    percent_missing: 0.0
    error_count: 0.0
    empty_count: 0.0
    unique_values: 5


ColumnProfile:
    column_name: pickup_datetime
    type: FieldType.STRING

    min: 2009-01-01 20:52:58
    max: 2013-08-31 23:59:14
    count: 7158.0
    missing_count: 0.0
    not_missing_count: 7158.0
    percent_missing: 0.0
    error_count: 0.0
    empty_count: 0.0
    unique_values: >1000


ColumnProfile:
    column_name: dropoff_datetime
    type: FieldType.STRING

    min: 2009-01-01 21:14:00
    max: 2013-09-01 00:25:07
    count: 7158.0
    missing_count: 0.0
    not_missing_count: 7158.0
    percent_missing: 0.0
    error_count: 0.0
    empty_count: 0.0
    unique_values: >1000


ColumnProfile:
    column_name: store_forward
    type: FieldType.STRING

    min: N
    max: Y
    count: 7158.0
    missing_count: 198.0
    not_mi

In [25]:
tmp_df = (combined_df
    .derive_column_by_example(
        source_columns="pickup_date", 
        new_column_name="pickup_weekday", 
        example_data=[("2009-01-04", "Sunday"), ("2013-08-22", "Thursday")]
    )
    .derive_column_by_example(
        source_columns="dropoff_date",
        new_column_name="dropoff_weekday",
        example_data=[("2013-08-22", "Thursday"), ("2013-11-03", "Sunday")]
    )
          
    .split_column_by_example(source_column="pickup_time")
    .split_column_by_example(source_column="dropoff_time")
    # the following two split_column_by_example calls reference the generated column names from the above two calls
    .split_column_by_example(source_column="pickup_time_1")
    .split_column_by_example(source_column="dropoff_time_1")
    .drop_columns(columns=[
        "pickup_date", "pickup_time", "dropoff_date", "dropoff_time", 
        "pickup_date_1", "dropoff_date_1", "pickup_time_1", "dropoff_time_1"
    ])
          
    .rename_columns(column_pairs={
        "pickup_date_2": "pickup_month",
        "pickup_date_3": "pickup_monthday",
        "pickup_time_1_1": "pickup_hour",
        "pickup_time_1_2": "pickup_minute",
        "pickup_time_2": "pickup_second",
        "dropoff_date_2": "dropoff_month",
        "dropoff_date_3": "dropoff_monthday",
        "dropoff_time_1_1": "dropoff_hour",
        "dropoff_time_1_2": "dropoff_minute",
        "dropoff_time_2": "dropoff_second"
    }))

tmp_df.head(5)

ValueError: Can't split column. Provide or update examples.

In [26]:
tmp_df = tmp_df.drop_columns(columns=["pickup_datetime", "dropoff_datetime"])

In [27]:
type_infer = tmp_df.builders.set_column_types()
type_infer.learn()
type_infer

Column types conversion candidates:
'vendor': [FieldType.STRING],
'store_forward': [FieldType.BOOLEAN],
'pickup_longitude': [FieldType.DECIMAL],
'pickup_latitude': [FieldType.DECIMAL],
'dropoff_longitude': [FieldType.DECIMAL],
'dropoff_latitude': [FieldType.DECIMAL],
'passengers': [FieldType.INTEGER],
'distance': [FieldType.DECIMAL],
'cost': [FieldType.DECIMAL]

In [28]:
tmp_df = type_infer.to_dataflow()
tmp_df.get_profile()

ColumnProfile:
    column_name: vendor
    type: FieldType.STRING

    min: 1
    max: VTS
    count: 7158.0
    missing_count: 0.0
    not_missing_count: 7158.0
    percent_missing: 0.0
    error_count: 0.0
    empty_count: 0.0
    unique_values: 5


ColumnProfile:
    column_name: store_forward
    type: FieldType.BOOLEAN

    min: False
    max: True
    count: 7158.0
    missing_count: 198.0
    not_missing_count: 6960.0
    percent_missing: 0.027661357921207042
    error_count: 0.0
    empty_count: 0.0
    unique_values: 3


ColumnProfile:
    column_name: pickup_longitude
    type: FieldType.DECIMAL

    min: -74.07815551757812
    max: -73.73648071289062
    count: 7158.0
    missing_count: 0.0
    not_missing_count: 7158.0
    percent_missing: 0.0
    error_count: 0.0
    empty_count: 0.0
    unique_values: >1000


    Quantiles (est.):
         0.1%: -74.07800495910645
           1%: -73.9638987099072
           5%: -73.96399193619052
          25%: -73.94948645963127
        

In [29]:
tmp_df = tmp_df.filter(dprep.col("distance") > 0)
tmp_df = tmp_df.filter(dprep.col("cost") > 0)

In [31]:
import azureml.dataprep as dprep

In [32]:
import os
file_path = os.path.join(os.getcwd(), "dflows.dprep")

dflow_prepared = tmp_df
package = dprep.Package([dflow_prepared])
package.save(file_path)

AttributeError: module 'azureml.dataprep' has no attribute 'Package'

In [33]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
import logging

In [34]:
ws = Workspace.from_config()
# choose a name for the run history container in the workspace
experiment_name = 'automated-ml-regression'
# project folder
project_folder = './automated-ml-regression'

import os

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

SDK version        1.0.69                              
Subscription ID    c46a9435-c957-4e6c-a0f4-b9a597984773
Workspace          eastus_automl_ws                    
Resource Group     eastus_automl_rg                    
Location           eastus2                             
Project Directory  ./automated-ml-regression

In [35]:
import azureml.dataprep as dprep

file_path = os.path.join(os.getcwd(), "dflows.dprep")

package_saved = dprep.Package.open(file_path)
dflow_prepared = package_saved.dataflows[0]
dflow_prepared.get_profile()

AttributeError: module 'azureml.dataprep' has no attribute 'Package'

In [36]:
dflow_X = dflow_prepared.keep_columns(['pickup_weekday','pickup_hour', 'distance','passengers', 'vendor'])
dflow_y = dflow_prepared.keep_columns('cost')

In [37]:
from sklearn.model_selection import train_test_split


x_df = dflow_X.to_pandas_dataframe()
y_df = dflow_y.to_pandas_dataframe()

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)
# flatten y_train to 1d array
y_train.values.flatten()

array([ 5.5, 10.5,  8. , ..., 11. , 33.5,  6.7])

In [38]:
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 30,
    "primary_metric" : 'spearman_correlation',
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 5
}

In [39]:
from azureml.train.automl import AutoMLConfig

# local compute 
automated_ml_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automated_ml_errors.log',
                             path = project_folder,
                             X = x_train.values,
                             y = y_train.values.flatten(),
                             **automl_settings)

In [40]:
from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(automated_ml_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_0a5e82e2-ccdf-4961-a85a-3e8cb7f89fd7
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High Cardinality Feature Detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.

**************************************************************

In [41]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

ModuleNotFoundError: No module named 'azureml.widgets'

In [42]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

0     1     2  ...     27    28    29
explained_variance                     0.56  0.86  0.84   ...  0.86  0.85  0.86 
mean_absolute_error                    4.23  1.61  1.88   ...  1.59  1.79  1.56 
mean_absolute_percentage_error         46.72 16.92 19.56  ...  15.99 19.79 15.69
median_absolute_error                  3.24  0.89  1.15   ...  1.09  1.26  0.95 
normalized_mean_absolute_error         0.05  0.02  0.02   ...  0.02  0.02  0.02 
normalized_median_absolute_error       0.04  0.01  0.01   ...  0.01  0.02  0.01 
normalized_root_mean_squared_error     0.08  0.04  0.05   ...  0.04  0.05  0.04 
normalized_root_mean_squared_log_error 0.09  0.05  0.05   ...  0.05  0.05  0.05 
r2_score                               0.56  0.86  0.84   ...  0.86  0.85  0.86 
root_mean_squared_error                6.36  3.59  3.84   ...  3.56  3.66  3.51 
root_mean_squared_log_error            0.41  0.22  0.23   ...  0.21  0.22  0.20 
spearman_correlation                   0.90  0.94  0.94   ...  0.95  0.

In [43]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automated-ml-regression,
Id: AutoML_0a5e82e2-ccdf-4961-a85a-3e8cb7f89fd7_28,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('pref...571428571427, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]))]),
          stddev=None)

In [44]:
description = 'Automated Machine Learning Model'
tags = None
local_run.register_model(description=description, tags=tags)
print(local_run.model_id) # Use this id to deploy the model as a web service in Azure

AutoML0a5e82e2cbest

In [45]:
y_predict = fitted_model.predict(x_test.values) 
print(y_predict[:10])

[13.77905083 30.93153336 11.84446908 33.21562784 13.65723075 18.05178827
 20.21589193  9.85561734 10.70397877 10.21175713]

In [46]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(14, 10))
ax1 = fig.add_subplot(111)

distance_vals = [x[4] for x in x_test.values]
y_actual = y_test.values.flatten().tolist()

ax1.scatter(distance_vals[:100], y_predict[:100], s=18, c='b', marker="s", label='Predicted')
ax1.scatter(distance_vals[:100], y_actual[:100], s=18, c='r', marker="o", label='Actual')

ax1.set_xlabel('distance (mi)')
ax1.set_title('Predicted and Actual Cost/Distance')
ax1.set_ylabel('Cost ($)')

plt.legend(loc='upper left', prop={'size': 12})
plt.rcParams.update({'font.size': 14})
plt.show()

IndexError: index 4 is out of bounds for axis 0 with size 3

In [47]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

NameError: name 'y_actual' is not defined

In [ ]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1
    
    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val
    
mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)